MLP (Multi Layer Perceptron) Classifier is the feedforward artificial neural network implementation of Scikit-Learn library.

MLPClassifier is used to predict whether a patient has diabetes based on a set of diagnostics.

Outline of the work is as follows:

* Load Data
* Split Data
* Visualization and Outlier Check
* Standardization
* Correlation Analysis
* Feature Importance
* Train MLP
* Test MLP

In [ ]:
import numpy as np
import itertools
import os
import pandas as pd
import seaborn as sea

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import RFE

import xgboost as xgb
from xgboost import plot_importance

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
sea.set_style("darkgrid")

## Load Data

Pima Indians Diabetes dataset is used.

In [ ]:
data = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/"
                   "diabetes.csv")

data.head(10).style.set_precision(2). \
                    set_properties(**{"min-width": "60px"}). \
                    set_properties(**{"color": "#111111"}). \
                    set_properties(**{"text-align": "center"}). \
                    set_table_styles([
                          {"selector": "th",
                           "props": [("font-weight", "bold"),
                                     ("font-size", "12px"),
                                     ("text-align", "center")]},
                          {"selector": "tr:nth-child(even)",
                           "props": [("background-color", "#f2f2f2")]},
                          {"selector": "tr:nth-child(odd)",
                           "props": [("background-color", "#fdfdfd")]},
                          {"selector": "tr:hover",
                           "props": [("background-color", "#bcbcbc")]}])

There are 8 features and Outcome is the target variable.

* Pregnancies
* Glucose
* BloodPressure
* SkinThickness
* Insulin
* BMI (Body mass index)
* DiabetesPedigreeFunction
* Age

Features are assigned to data_X and corresponding labels to data_Y. Pandas info shows column (feature) data types and number of non-null values.

In [ ]:
# disable SettingWithCopyWarning messages
pd.options.mode.chained_assignment = None

data_X = data.loc[:, data.columns != "Outcome"]
data_Y = data[["Outcome"]]

print("data_X info:\n")
data_X.info()
print("\ndata_Y info:\n")
data_Y.info()

Dataset has 768 rows (training samples). All features are numeric. And it seems there aren't any missing values. Here, one point requires extra attention. In your dataset, missing values may be encoded other than **NaN** or **NA**. For example, if you go and change some cells in dataset to **Na** or **NAN** and load it again and check info, **Pandas** won't recognize them as null values. So, **Pandas** may not detect these cells as missing. In our case, dataset is small and a short visual inspection reveals no missing values. But there is another point. Some cells may be encoded with an unlikely input. For example, 0 may indicate a missing value for a numeric feature. We will return this issue later.

Number of classes and their distribution is inspected in below cell.

In [ ]:
data_Y["Outcome"].value_counts()

There are two classes as expected, a patient has diabetes or doesn't have.

## Split Data

Dataset is divided into train and test sets. We use stratify parameter of train_test_split function to get the same class distribution across train and test sets.

In [ ]:
train_X, test_X, train_Y, test_Y = train_test_split(data_X, data_Y,
                                                    test_size=0.2,
                                                    stratify=data_Y,
                                                    random_state=0)

train_X.reset_index(drop=True, inplace=True);
test_X.reset_index(drop=True, inplace=True);
train_Y.reset_index(drop=True, inplace=True);
test_Y.reset_index(drop=True, inplace=True);

## Visualization and Outlier Check

Outliers degrade the learning performance. Outlier analysis is performed for each feature one-by-one. We use quartile analysis for outlier detection. For each feature, there are two plots below. Distribution of feature is on the left. Box plot of the same feature is on the right. Both of them are analyzed together to get an idea about the outliers. From this point on, lower whisker of the boxplot is denoted as LW and upper whisker is denoted as UW.

In order to draw a boxplot, feature data is divided into four. Three cut points are needed. These points are lower quartile (or first quartile), median (or second quartile) and upper quartile (or third quartile). First quartile is the median of the data lower than second quartile. Third quartile is the median of the data greater than the second quartile. Interquartile range (IQR) is found subtracting lower quartile from upper quartile. Outliers are determined using lower, upper quartiles and IQR.

In [ ]:
def plots(feature):
    fig = plt.figure(constrained_layout = True, figsize=(10,3))
    gs = gridspec.GridSpec(nrows=1, ncols=4, figure=fig)

    ax1 = fig.add_subplot(gs[0,:3])    
    sea.distplot(train_X.loc[train_Y["Outcome"]==0,feature],
                 kde = False, color = "#004a4d", norm_hist=False,
                 hist_kws = dict(alpha=0.8), bins=40,
                 label="Not Diabetes", ax=ax1);
    sea.distplot(train_X.loc[train_Y["Outcome"]==1,feature],
                 kde = False, color = "#7d0101", norm_hist=False,
                 hist_kws = dict(alpha=0.6), bins=40,
                 label="Diabetes", ax=ax1);
    ax2 = fig.add_subplot(gs[0,3])    
    sea.boxplot(train_X[feature], orient="v", color = "#989100",
                width = 0.2, ax=ax2);
    
    ax1.legend(loc="upper right");

### Feature 0 - Pregnancies

In [ ]:
plots("Pregnancies")

For Pregnancies feature, there are some measurements above upper whisker. These are rare events. We replace them with 95th quantile.

In [ ]:
Q1 = train_X["Pregnancies"].quantile(0.25)
Q3 = train_X["Pregnancies"].quantile(0.75)
q95th = train_X["Pregnancies"].quantile(0.95)
IQR = Q3 - Q1
UW = Q3 + 1.5*IQR

train_X["Pregnancies"] = np.where(train_X["Pregnancies"] > UW,
                                  q95th, train_X["Pregnancies"])

### Feature 1 - Glucose

In [ ]:
plots("Glucose")

There are some 0 values for **Glucose**. We can deem 0 values as placeholder for missing data. So we replace them with median.

In [ ]:
med = train_X["Glucose"].median()
train_X["Glucose"] = np.where(train_X["Glucose"] == 0, med, train_X["Glucose"])

### Feature 2 - BloodPressure

In [ ]:
plots("BloodPressure")

There are some 0 values for **BloodPressure** which is unlikely. So we replace them with median. Also, we replace values lower than LW (except zeros) with 5th quantile and replace values greater than UW with 95th quantile.

In [ ]:
med = train_X["BloodPressure"].median()
q5th = train_X["BloodPressure"].quantile(0.05)
q95th = train_X["BloodPressure"].quantile(0.95)
Q1 = train_X["BloodPressure"].quantile(0.25)
Q3 = train_X["BloodPressure"].quantile(0.75)
IQR = Q3 - Q1
LW = Q1 - 1.5*IQR
UW = Q3 + 1.5*IQR

train_X["BloodPressure"] = np.where(train_X["BloodPressure"] == 0,
                                    med, train_X["BloodPressure"])
train_X["BloodPressure"] = np.where(train_X["BloodPressure"] < LW,
                                    q5th, train_X["BloodPressure"])
train_X["BloodPressure"] = np.where(train_X["BloodPressure"] > UW,
                                    q95th, train_X["BloodPressure"])

### Feature 3 - SkinThickness

In [ ]:
plots("SkinThickness")

There are some 0 values for **SkinThickness** which is unlikely. So we replace them with median. Also, we replace values greater than UW with 95th quantile.

In [ ]:
med = train_X["SkinThickness"].median()
q95th = train_X["SkinThickness"].quantile(0.95)
Q1 = train_X["SkinThickness"].quantile(0.25)
Q3 = train_X["SkinThickness"].quantile(0.75)
IQR = Q3 - Q1
UW = Q3 + 1.5*IQR

train_X["SkinThickness"] = np.where(train_X["SkinThickness"] == 0,
                                    med, train_X["SkinThickness"])
train_X["SkinThickness"] = np.where(train_X["SkinThickness"] > UW,
                                    q95th, train_X["SkinThickness"])

### Feature 4 - Insulin

In [ ]:
plots("Insulin")

There are some 0 values for **Insulin** which is unlikely. So we replace them with 60th quantile becuse median is 0. Also, we replace values greater than UW with 95th quantile.

In [ ]:
q60th = train_X["Insulin"].quantile(0.60)
q95th = train_X["Insulin"].quantile(0.95)
Q1 = train_X["Insulin"].quantile(0.25)
Q3 = train_X["Insulin"].quantile(0.75)
IQR = Q3 - Q1
UW = Q3 + 1.5*IQR

train_X["Insulin"] = np.where(train_X["Insulin"] == 0,
                              q60th, train_X["Insulin"])
train_X["Insulin"] = np.where(train_X["Insulin"] > UW,
                              q95th, train_X["Insulin"])

### Feature 5 - BMI

In [ ]:
plots("BMI")

There are some 0 values for **BMI**. We replace them with median. Also, we replace values greater than UW with q95th.

In [ ]:
med = train_X["BMI"].median()
q95th = train_X["BMI"].quantile(0.95)
Q1 = train_X["BMI"].quantile(0.25)
Q3 = train_X["BMI"].quantile(0.75)
IQR = Q3 - Q1
UW = Q3 + 1.5*IQR

train_X["BMI"] = np.where(train_X["BMI"] == 0,
                          med, train_X["BMI"])
train_X["BMI"] = np.where(train_X["BMI"] > UW,
                          q95th, train_X["BMI"])

### Feature 6 - DiabetesPedigreeFunction

In [ ]:
plots("DiabetesPedigreeFunction")

We replace values greater than UW with 95th quantile.

In [ ]:
q95th = train_X["DiabetesPedigreeFunction"].quantile(0.95)
Q1 = train_X["DiabetesPedigreeFunction"].quantile(0.25)
Q3 = train_X["DiabetesPedigreeFunction"].quantile(0.75)
IQR = Q3 - Q1
UW = Q3 + 1.5*IQR

train_X["DiabetesPedigreeFunction"] = np.where(
                        train_X["DiabetesPedigreeFunction"] > UW,
                        q95th, train_X["DiabetesPedigreeFunction"])

### Feature 7 - Age

In [ ]:
plots("Age")

There are some measurements above UW due to rare events. We replace them with 95th quantile.

In [ ]:
q95th = train_X["Age"].quantile(0.95)
Q1 = train_X["Age"].quantile(0.25)
Q3 = train_X["Age"].quantile(0.75)
IQR = Q3 - Q1
UW = Q3 + 1.5*IQR

train_X["Age"] = np.where(train_X["Age"] > UW,
                          q95th, train_X["Age"])

## Standardization

To increase the learning performance, input features are standardized. Mean and standard deviation of the feature are computed. Then, mean is subtracted from each sample of the feature and result is divided by standard deviation. The aim is to transform the feature to have mean of 0 and standard deviation of 1. **StandardScaler** of **scikit-learn** is used. A **StandardScaler** is fit to the feature in **train_X**, then this scaler transforms the same feature in **train_X** and **test_X**.

In [ ]:
feature_names = train_X.columns

scaler = StandardScaler()

# fit to train_X
scaler.fit(train_X)

# transform train_X
train_X = scaler.transform(train_X)
train_X = pd.DataFrame(train_X, columns = feature_names)

# transform test_X
test_X = scaler.transform(test_X)
test_X = pd.DataFrame(test_X, columns = feature_names)

## Correlation Analysis

Linear correlations between features and also between features and output are computed. **Pandas corr** function is used to compute correlation matrix and **Seaborn heatmap** is used for plotting.

In [ ]:
corr_matrix = pd.concat([train_X, train_Y], axis=1).corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=np.bool))

plt.figure(figsize=(10,8))
sea.heatmap(corr_matrix,annot=True, fmt=".4f",
            vmin=-1, vmax=1, linewidth = 1,
            center=0, mask=mask,cmap="RdBu_r");

Correlation matrix shows that there are mild correlations between **SkinThickness-BMI** and **Age-Pregnancies**. **Outcome** has the highest linear correlation with **Glucose**.

## Feature Importance

Feature importance analysis methods can be divided into 3 broad categories:

* Filter method: Calculating a metric like correlation coefficient between each feature and output separately as we did above. In this method all features are evaluated independently.

* Embedded methods: Methods like logistic regression or linear regression learn the coefficients that multiply each feature. The magnitude of the coefficients are associated with the importance of the features. Also, tree based methods like random forests or gradient tree boosting learn feature importances during training process. In embedded methods, all features are evaluated jointly.

* Wrapper methods: Basically you have an estimator and you train this estimator with the subsets of features. The subset giving the best score is selected and other features are eliminated.

In [ ]:
# logistic regression with l1 penalty - embedded method
lr1 = LogisticRegression(random_state=0, penalty="l1",
            class_weight="balanced", solver="saga", max_iter=5000)
lr1.fit(train_X, train_Y.values.ravel())

lr1.coef_

In [ ]:
# logistic regression with l2 penalty - embedded method
lr2 = LogisticRegression(random_state=0, penalty="l2",
            class_weight="balanced", solver="saga", max_iter=5000)
lr2.fit(train_X, train_Y.values.ravel())

lr2.coef_

In [ ]:
# xgboost - embedded method
gb = xgb.XGBClassifier(booster="gbtree",
                      learning_rate=0.15,
                      max_depth=4,
                      n_estimators=20,
                      random_state=0)
gb.fit(train_X, train_Y.values.ravel())

plot_importance(gb);

In [ ]:
# recursive feature elimination with logistic regression - wrapper method
lr3 = LogisticRegression(random_state=0, penalty="l2",
            class_weight="balanced", solver="saga", max_iter=5000)
rfe_lr3 = RFE(lr3, n_features_to_select=7)
rfe_lr3.fit(train_X, train_Y.values.ravel())

print(rfe_lr3.support_)

First three analysis mark BloodPressure, SkinThickness and Insulin as the least important features. Note that these features have very low correlation with Outcome. RFE marks SkinThickness as the least important.

In [ ]:
train_X.drop("SkinThickness", axis=1, inplace=True)
test_X.drop("SkinThickness", axis=1, inplace=True)

## Train MLP

MLPClassifier with single hidden layer is used for diabetes prediction.

In [ ]:
clf = MLPClassifier(solver="adam", max_iter=5000, activation = "relu",
                    hidden_layer_sizes = (12),                      
                    alpha = 0.01,
                    batch_size = 64,
                    learning_rate_init = 0.001,
                    random_state=2)

clf.fit(train_X, train_Y.values.ravel());

## Test MLP

In [ ]:
print(classification_report(test_Y, clf.predict(test_X),
                            digits = 4,
                            target_names=["Not Diabetes",
                                          "Diabetes"]))